In [14]:
from ipywebrtc import CameraStream, ImageRecorder
import ipywidgets as widgets
from ipywidgets import VBox, HBox

import PIL.Image
import io

from fastai2.vision.all import *

In [15]:
#setup learner
inf = load_learner('/storage/lr_hard_center_resnet34_512_6epochs.pkl')

In [12]:
# setup gui elements

camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })

image_recorder = ImageRecorder(format='png', stream=camera)
image_recorder.recording=True

btn_classify = widgets.Button(description="Click Me!")
lbl_result = widgets.Output()

# callbacks
def classify(btn):
    global image_recorder
    
    im = PILImage.create(io.BytesIO(image_recorder.image.value))
    #im = PILImage.frombytes(image_recorder.image.value)
    prediction = inf.predict(im)
    lbl,_,_ = prediction
    
    if not lbl: msg = 'Unknown Direction'
    else:
        if 'Center' in lbl:
            msg = 'Center'
        elif 'Left' in lbl:
            msg = 'Right'
        elif 'Right' in lbl:
            msg = 'Left'
    
    lbl_result.clear_output()
    
    with lbl_result:
        print(prediction)

        
btn_classify.on_click(classify)

In [13]:
HBox([camera, VBox([image_recorder, HBox([btn_classify, lbl_result])])])